# Bathymetric Data Visualization
### Objective(s) for this notebook:
- Acquire contour line features for entire Mediterranean
- Use plots to understand the depth lines
- Possibly export vector data as SVGs to mess around with in other programs

In [1]:
# You can find more detail on these libraries by consulting my install script or
# visiting their page on CRAN.
library(XML)
library(ncdf4)
library(sp)
library(terra)
library(downloader)
library(ggplot2)
library(directlabels)
library(rasterVis)
library(raster)
library(gdalUtilities)

terra 1.7.55

Loading required package: lattice



In [2]:
# Set a bounding box for the entire Mediterranean:
xmin <- -6
xmax <- 37
ymin <- 29
ymax <- 46

## Loading features from WFS:

In [3]:
get_bathymetry_contours <- function(xmin = 22.5, xmax = 25, ymin = 37.5, ymax = 40) {
  feature_type <- "emodnet:contours"
  bounding_box <- paste(xmin, ymin, xmax, ymax, sep = ",")
  url <- paste("https://ows.emodnet-bathymetry.eu/wfs?service=WFS&version=1.0.0&request=GetFeature&typeName=",
               feature_type, "&OutputFormat=json&bbox=", bounding_box, sep = "")
  print(paste("FULL Request URL:", url))
  # a function in terra for reading a GeoJSON file into terra's S4 vector
  layer <- terra::vect(url) # , layer = "OGRGeoJSON"
  contours <- terra::geom(layer) # turning vect into geom matri
  contours <- as.data.frame(contours) # adding a data frame to geom matrix
  return(contours)
}

In [4]:
contours <- get_bathymetry_contours(xmin, xmax, ymin, ymax)

[1] "FULL Request URL: https://ows.emodnet-bathymetry.eu/wfs?service=WFS&version=1.0.0&request=GetFeature&typeName=emodnet:contours&OutputFormat=json&bbox=-6,29,37,46"


## Loading features from cached `contours` object:

In [8]:
# Uncomment to save:
# saveRDS(contours, file = "./rcache/contours.rdat")

In [ ]:
# Uncomment to load:
# contours <- readRDS(file = "./rcache/contours.rdat")

## Exploring data:

In [9]:
head(contours)

,geom,part,x,y,hole
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,0.3666,38.6344,0
2,1,1,0.3666,38.6343,0
3,1,1,0.3668,38.6341,0
4,1,1,0.3670,38.6338,0
5,1,1,0.3672,38.6336,0
6,1,1,0.3674,38.6333,0


`part` and `hole` are always 1 & 0 respectively.  I'm not sure what their purpose is.  Somewhere along the way, the elevation component of the data was lost.

In [13]:
dim.data.frame(contours)

[1] 4524454       5

## Plotting:

# Pause:
I think this strategy of fetching contour lines isn't going to work for my purposes.  The contours stored by EMODnet are insufficient, because they go in irregular intervals (50,100,200,500,1000,2000,5000 meters).  I think I'm going to have to make my own contour lines with the raster data, which really sucks.  The reaster data is more complete, but much larger in size, so I will have to sort of download it in tiles, stitch the tiles together, and then create vectors from that.

**I guess the approach should be as follows:**
- Download a small tile from _WCS_
- attempt to make successive contours from it with a contour making function, which exports a list or set of contours in vector format
- Then, try to stich together tiles to create the entire Mediterranean raster
- Acquire, plot, and export contours for this stitch